In [ ]:
# Step1: Generate Head Dataset and Velocity Dataset
import numpy as np
import os
import pandas as pd
import h5py
from numba import jit, prange

# Parameter settings
params = {
    'p': 3, 'q': 3, 'HRx': 6, 'HLx': 3, 'HRy': 3, 'HLy': 0,
    'LX': 10000, 'LY': 10000, 'D': -2000, 'a': 2/3, 'b': 1/3,
    'TauD': 1
}

# Extract parameters
p, q, HRx, HLx, HRy, HLy = params['p'], params['q'], params['HRx'], params['HLx'], params['HRy'], params['HLy']
LX, LY, D, a, b, TauD = params['LX'], params['LY'], params['D'], params['a'], params['b'], params['TauD']

# Define the mesh grid
mesh_x, mesh_y, mesh_z = 101, 101, 101
x_range = np.linspace(0, LX, mesh_x)
y_range = np.linspace(0, LY, mesh_y)
z_range = np.linspace(D, 0, mesh_z)

# Precompute cosine values
b1 = np.cos(np.pi * x_range / LX)
b2 = np.cos(p * np.pi * x_range / LX)
b3 = np.cos(np.pi * y_range / LY)
b4 = np.cos(q * np.pi * y_range / LY)

@jit(nopython=True, parallel=True)
def compute_J(mesh_x, mesh_y, mesh_z, x_range, y_range, z_range, HRx, HLx, HRy, HLy, LX, LY, D, a, b, tp, TauD, b1, b2, b3, b4, c1, c2, c3, c4, e1, e2):
    """Calculate the J matrix and related variables using the provided parameters."""
    J = np.zeros((mesh_x, mesh_y, mesh_z))
    for i in prange(mesh_x):
        for j in prange(mesh_y):
            for k in prange(mesh_z):
                # Calculate potential components hA, hB, and hC
                hA = HRx + HRy - HRx * b1[i] * np.cosh(np.pi * (z_range[k] - D) / LX) / c1 - HRy * b3[j] * np.cosh(np.pi * (z_range[k] - D) / LY) / c2
                hB = HLx + HLy - HLx * b2[i] * np.cosh(p * np.pi * (z_range[k] - D) / LX) / c3 - HLy * b4[j] * np.cosh(q * np.pi * (z_range[k] - D) / LY) / c4
                g1, g2, g3 = 0.0, 0.0, 0.0
                for n in range(1, 1000):  
                    k1, an = (-1) ** (n - 1), (2 * n - 1) * np.pi / 2
                    b0 = np.cos(an * (z_range[k] - D) / D)
                    f1 = D * (k1 * an * b0 * (1 / an**2 + (an**2 * e1 + 2 * np.pi * TauD * e2) / (an**4 + 4 * np.pi**2 * TauD**2)))
                    g1 += f1
                    k2, d1 = (-1) ** n, an**2 + (p * np.pi * D / LX)**2
                    f2 = D * (k2 * an * b0 * (1 / d1 + (d1 * e1 + 2 * np.pi * TauD * e2) / (d1**2 + 4 * np.pi**2 * TauD**2)))
                    g2 += f2
                    k3, d2 = (-1) ** n, an**2 + (q * np.pi * D / LY)**2
                    f3 = D * (k3 * an * b0 * (1 / d2 + (d2 * e1 + 2 * np.pi * TauD * e2) / (d2**2 + 4 * np.pi**2 * TauD**2)))
                    g3 += f3
                hC = (HLx + HLy) * g1 / D + HLx * b2[i] * g2 / D + HLy * b4[j] * g3 / D
                J[i, j, k] = hA + a * hB + b * hC
    return J

def calculate_fields(tp):
    """Calculate the potential field (J), velocity components (vx, vy, vz), and velocity magnitude (UU) for a given time point (tp)."""
    e1 = np.cos(2 * np.pi * tp)
    e2 = np.sin(2 * np.pi * tp)

    # Precompute cosh values for efficiency
    c1 = np.cosh(np.pi * D / LX)
    c2 = np.cosh(np.pi * D / LY)
    c3 = np.cosh(p * np.pi * D / LX)
    c4 = np.cosh(q * np.pi * D / LY)

    J = compute_J(mesh_x, mesh_y, mesh_z, x_range, y_range, z_range, HRx, HLx, HRy, HLy, LX, LY, D, a, b, tp, TauD, b1, b2, b3, b4, c1, c2, c3, c4, e1, e2)

    # Compute the gradient of J
    dx, dy, dz = x_range[1] - x_range[0], y_range[1] - y_range[0], z_range[1] - z_range[0]
    grad_x, grad_y, grad_z = np.gradient(J, dx, dy, dz)

    # Calculate the velocity components as the negative gradient of J
    vx, vy, vz = -grad_x, -grad_y, -grad_z

    # Calculate the magnitude of the velocity field
    UU = np.sqrt(vx**2 + vy**2 + vz**2)

    # Hydraulic head (H) is equivalent to J
    H = J

    return H, vx, vy, vz, UU

def save_data_hdf5(filename, J, vx, vy, vz, UU, H, x_range, y_range, z_range):
    """Save the computed data into an HDF5 file."""
    with h5py.File(filename, 'w') as f:
        f.create_dataset('J', data=J)
        f.create_dataset('vx', data=vx)
        f.create_dataset('vy', data=vy)
        f.create_dataset('vz', data=vz)
        f.create_dataset('UU', data=UU)
        f.create_dataset('H', data=H)
        f.create_dataset('x_range', data=x_range)
        f.create_dataset('y_range', data=y_range)
        f.create_dataset('z_range', data=z_range)

def save_data_tecplot(filename, vx, vy, vz, UU, J, x_range, y_range, z_range):
    """Save the computed data into a Tecplot-compatible .dat file."""
    # Apply mask for z_range < -20
    mask = z_range < -20

    x, y, z = np.meshgrid(x_range, y_range, z_range, indexing='ij')
    
    # Flatten and filter data
    x = x[:, :, mask].flatten()
    y = y[:, :, mask].flatten()
    z = z[:, :, mask].flatten()
    vx = vx[:, :, mask].flatten()
    vy = vy[:, :, mask].flatten()
    vz = vz[:, :, mask].flatten()
    UU = UU[:, :, mask].flatten()
    J = J[:, :, mask].flatten()

    # Stack data column-wise
    data = np.column_stack((x, y, z, vx, vy, vz, UU, J))

    # Sort data by x, then y, then z
    data = data[np.lexsort((x, y, z))]

    with open(filename, 'w') as f:
        f.write('TITLE = "Velocity, Speed and Head Data"\n')
        f.write('VARIABLES = "X", "Y", "Z", "VX", "VY", "VZ", "UU", "H"\n')
        f.write(f'ZONE T="Flow Field", I={len(x_range)}, J={len(y_range)}, K={len(z_range[mask])}, F=POINT\n')
        np.savetxt(f, data, fmt='%.6f', delimiter=' ')

# Setup output directory
current_working_dir = os.getcwd()
result_folder = os.path.join(current_working_dir, 'results')
os.makedirs(result_folder, exist_ok=True)

# Loop over time points (tp) and compute, save results
for tp in np.arange(0, 1, 0.25):
    H, vx, vy, vz, UU = calculate_fields(tp)

    # Construct file names for the current time point
    hdf5_filename = os.path.join(result_folder, f'head_and_velocity_tp_{tp:.2f}.h5')
    tecplot_filename = os.path.join(result_folder, f'head_and_velocity_tp_{tp:.2f}.dat')
    
    # Save data to HDF5 format
    save_data_hdf5(hdf5_filename, H, vx, vy, vz, UU, H, x_range, y_range, z_range)
    
    # Save data to Tecplot .dat format
    save_data_tecplot(tecplot_filename, vx, vy, vz, UU, H, x_range, y_range, z_range)

    # Print progress
    print(f"Completed processing for tp = {tp:.2f}")

# End of processing
print("All time points processed and data saved.")


In [ ]:
# Step2: Finding Out (Pseudo-)stagnation Points/Lines
import os
from pathlib import Path
import numpy as np
import pandas as pd
import h5py
import plotly.graph_objects as go
import plotly.io as pio
from scipy.interpolate import griddata
from scipy.ndimage import minimum_filter
from concurrent.futures import ThreadPoolExecutor

# Get the current working directory
script_dir = Path.cwd()

# Define the range boundaries
x_min, x_max = 0, 10000
y_min, y_max = 0, 10000
z_min, z_max = -2000, 0
threshold = 1e-3

# Define a function to filter points that are at least 200 meters apart
def filter_points(points):
    filtered = []
    for point in points:
        if all(np.linalg.norm(np.array(point) - np.array(p)) >= 200 for p in filtered):
            filtered.append(point)
    return filtered

# Process each y-slice
def process_slice_y(y, X_filtered, Y_filtered, Z_filtered, vx_filtered, vy_filtered, vz_filtered, UU_filtered):
    slice_indices = np.where(Y_filtered == y)[0]
    if len(slice_indices) == 0:
        return []

    points = np.array([X_filtered[slice_indices], Z_filtered[slice_indices]]).T
    vx_values = vx_filtered[slice_indices]
    vy_values = vy_filtered[slice_indices]
    vz_values = vz_filtered[slice_indices]
    UU_values = UU_filtered[slice_indices]

    # Create a grid for interpolation
    grid_x, grid_z = np.mgrid[x_min:x_max:200j, z_min:z_max:200j]
    grid_vx = griddata(points, vx_values, (grid_x, grid_z), method='linear')
    grid_vy = griddata(points, vy_values, (grid_x, grid_z), method='linear')
    grid_vz = griddata(points, vz_values, (grid_x, grid_z), method='linear')
    grid_UU = griddata(points, UU_values, (grid_x, grid_z), method='linear')

    # Identify local minima in UU
    UU_local_min = (grid_UU == minimum_filter(grid_UU, size=50))
    UU_local_min_points_slice = []

    # Collect points that meet the threshold condition
    for i in range(grid_x.shape[0]):
        for j in range(grid_x.shape[1]):
            if UU_local_min[i, j] and x_min + 1 <= grid_x[i, j] <= x_max - 1 and z_min + 1 <= grid_z[i, j] <= z_max - 1:
                if grid_UU[i, j] < threshold:
                    UU_local_min_points_slice.append((grid_x[i, j], y, grid_z[i, j], grid_UU[i, j]))

    return filter_points(UU_local_min_points_slice)

# Aggregate and filter points from all slices
def compare_and_filter_points(points):
    filtered = []
    for point in points:
        existing_point = next((p for p in filtered if np.linalg.norm(np.array(point[:3]) - np.array(p[:3])) < 1), None)
        if existing_point:
            if point[3] < existing_point[3]:
                filtered.remove(existing_point)
                filtered.append(point)
        else:
            filtered.append(point)
    return filtered

# Save points to a CSV file
def save_points_to_csv(points, file_name, header):
    csv_path = script_dir / 'results' / file_name
    np.savetxt(csv_path, points, delimiter=', ', header=header, comments='', fmt='%s')
    print(f"Found and saved {len(points)} points to {file_name}")

# Generate a .dat file from the CSV file
def process_csv_file(file_prefix):
    input_path = script_dir / 'results' / f'{file_prefix}.csv'
    df = pd.read_csv(input_path)

    # Remove any spaces in column names
    df.columns = df.columns.str.strip()
    #print(f"Column names: {df.columns.tolist()}")

    # Ensure required columns are present
    required_columns = ['x', 'y', 'z']
    actual_columns = df.columns.tolist()

    if not all(col in actual_columns for col in required_columns):
        print(f"Error: CSV file is missing required columns. Actual columns: {actual_columns}")
        return

    # Save the .dat file with required formatting
    output_path_dat = script_dir / 'results' / f'{file_prefix}.dat'
    df[['x', 'y', 'z']].to_csv(output_path_dat, sep='\t', index=False, header=False)

    with open(output_path_dat, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write('VARIABLES = "X", "Y", "Z"\n')
        f.write(f"ZONE T='{file_prefix}'\n")
        f.write(f'I={len(df)}, J=1, K=1, F=POINT\n')
        f.write(content)
    
    print(f'File generated: {output_path_dat}')

# Generate a 3D scatter plot for visualization
def plot_points(all_points, title):
    fig = go.Figure()
    base_colors = ['#FF0000', '#FFA500', '#008000', '#87CEEB', '#00FFFF', '#0000FF', '#FF00CC', '#A52A2A', '#808080', '#000000']
    
    # Extend the color list if necessary
    num_colors_needed = len(all_points)
    colors = (base_colors * (num_colors_needed // len(base_colors) + 1))[:num_colors_needed]

    for tp, color in zip(sorted(all_points.keys()), colors):
        points = all_points[tp]
        if len(points) == 0:  # Check if points list is empty
            print(f"No points found for tp={tp:.2f}, skipping...")
            continue  # Skip this iteration if no points are found
        x, y, z, _ = zip(*points)
        fig.add_trace(go.Scatter3d(
            x=x,
            y=y,
            z=z,
            mode='markers',
            marker=dict(size=5, color=color),
            name=f'tp={tp}'
        ))

    # Update the layout of the figure
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            xaxis=dict(range=[x_min, x_max]),
            yaxis=dict(range=[y_min, y_max]),
            zaxis=dict(range=[z_min, z_max]),
            aspectmode='manual',
            aspectratio=dict(x=1, y=1, z=0.5)
        ),
        margin=dict(l=0, r=0, b=0, t=0)
    )

    return fig


# Main code execution
def main():
    all_points_y = {}
    for tp in np.arange(0, 1, 0.25):
        file_relative_path = Path('results', f'head_and_velocity_tp_{tp:.2f}.h5')
        file_path = script_dir / file_relative_path

        # Check if the HDF5 file exists
        if not file_path.exists():
            print(f"Error: File '{file_path}' does not exist!")
            continue

        try:
            # Load data from the HDF5 file
            with h5py.File(file_path, 'r') as f:
                x_range = f['x_range'][:]
                y_range = f['y_range'][:]
                z_range = f['z_range'][:]
                vx = f['vx'][:]
                vy = f['vy'][:]
                vz = f['vz'][:]
                UU = f['UU'][:]
            #print(f"File successfully loaded: {file_relative_path}")
        except Exception as e:
            #print(f"Error loading file: {e}")
            continue

        # Apply the filtering criteria to extract relevant data
        X, Y, Z = np.meshgrid(x_range, y_range, z_range, indexing='ij')
        filtered_indices = (
            (X >= x_min) & (X <= x_max) &
            (Y >= y_min) & (Y <= y_max) &
            (Z >= z_min) & (Z <= z_max)
        )
        X_filtered = X[filtered_indices]
        Y_filtered = Y[filtered_indices]
        Z_filtered = Z[filtered_indices]
        vx_filtered = vx[filtered_indices]
        vy_filtered = vy[filtered_indices]
        vz_filtered = vz[filtered_indices]
        UU_filtered = UU[filtered_indices]

        # Define y-slices for processing
        y_slices = np.arange(y_min, y_max+ 1, 100)
        UU_local_min_points_y = []

        # Use parallel processing for each y-slice
        with ThreadPoolExecutor() as executor:
            results_y = executor.map(lambda y: process_slice_y(y, X_filtered, Y_filtered, Z_filtered, vx_filtered, vy_filtered, vz_filtered, UU_filtered), y_slices)
            for res in results_y:
                UU_local_min_points_y.extend(res)

        # Filter and save the local minima points for each time step (tp)
        UU_local_min_points_filtered = compare_and_filter_points(UU_local_min_points_y)
        save_points_to_csv(UU_local_min_points_filtered, f'best_y_tp_{tp:.2f}.csv', 'x, y, z, UU\n')
        process_csv_file(f'best_y_tp_{tp:.2f}')
        all_points_y[tp] = UU_local_min_points_filtered

    # Plot the points for all time steps
    fig_best_y = plot_points(all_points_y, 'Best Points from Y Slices with Local Min UU')
    output_html_file = script_dir / 'results' / 'pseudostagline_Case2T_taud_1_tsum.html'
    pio.write_html(fig_best_y, file=output_html_file, auto_open=False)
    fig_best_y.show()

if __name__ == "__main__":
    main()


In [ ]:
import os
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from pathlib import Path

# Get the current working directory
script_dir = Path.cwd()

# Define the target directory for results
results_dir = script_dir / 'results'

# Create the results directory if it doesn't exist
if not results_dir.exists():
    results_dir.mkdir(parents=True, exist_ok=True)

# Define target y-values
y_targets = [0, 5000, 10000]

# Define the range of tp values
tp_range = np.arange(0, 1, 0.25)

# Create a dictionary to store points for each y_target at different tp values
y_points = {y: {} for y in y_targets}

# Iterate through all tp values
for tp in tp_range:
    file_path = results_dir / f'best_y_tp_{tp:.2f}.csv'
    
    # Check if the file exists
    if not file_path.exists():
        print(f"Error: File '{file_path}' does not exist!")
        continue

    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Filter points where y matches the target values, and x, z are within specified ranges
    for y_target in y_targets:
        # Use columns with leading spaces if they exist in the file
        df_y_target = df[(df[' y'] == y_target) & (df['x'].between(4000, 6000)) & (df[' z'].between(-1000, 0))]
        
        # Store the filtered points
        y_points[y_target][tp] = df_y_target[['x', ' z']].values

# Define a list of colors for different tp values
colors = [
    '#FF0000',  # Red
    '#FFA500',  # Orange
    '#008000',  # Green
    '#87CEEB',  # Sky Blue
    '#00FFFF',  # Cyan
    '#0000FF',  # Blue
    '#FF00CC',  # Magenta
    '#A52A2A',  # Brown
    '#808080',  # Gray
    '#000000',  # Black
]

# Create a plot for each y_target and save the data to CSV
for y_target in y_targets:
    fig = go.Figure()
    
    # Collect all points
    all_points = []
    
    # Iterate through each tp value and its corresponding color
    for idx, tp in enumerate(y_points[y_target].keys()):
        color = colors[idx % len(colors)]  # Use modulo to loop through colors
        
        points = y_points[y_target][tp]
        if len(points) > 0:
            fig.add_trace(go.Scatter(
                x=points[:, 0],  # x-coordinates
                y=points[:, 1],  # z-coordinates
                mode='markers',
                marker=dict(size=5, color=color),
                name=f'tp={tp:.2f}'
            ))
            
            # Add these points to the all_points list
            all_points.extend([(x, z, tp) for x, z in points])
    
    # Save the collected points to a CSV file
    csv_file_path = results_dir / f'stagnation_point_y_{y_target}.csv'
    df_all_points = pd.DataFrame(all_points, columns=['x', 'z', 'tp'])
    df_all_points.to_csv(csv_file_path, index=False)
    
    # Configure the layout of the plot
    fig.update_layout(
        title=f'Points for y={y_target}',
        xaxis_title='X',
        yaxis_title='Z',
        xaxis_range=[4000, 6000],  # Set x-axis range
        yaxis_range=[-1000, 0],    # Set z-axis range
        showlegend=True
    )
    
    # Save the figure as an HTML file
    html_file_path = results_dir / f'stagnation_point_in_y{y_target}.html'
    fig.write_html(html_file_path)
    
    # Display the plot
    fig.show()

In [ ]:
# Step3-1: Generate Critical Points Around (Pseudo-)Stagnation Points along 6 directions
import numpy as np
import pandas as pd
import logging
from pathlib import Path
import plotly.graph_objects as go
from collections import namedtuple

# Set up logging to track events during processing
logging.basicConfig(filename='processing.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Function to read the CSV file containing best y points
def read_best_y(file_prefix):
    input_path = Path('results') / f'{file_prefix}.csv'
    # Check if the file exists and is not empty
    if not input_path.exists() or input_path.stat().st_size == 0:
        logging.warning(f"File {input_path} does not exist or is empty.")
        return None
    return pd.read_csv(input_path)

# Function to calculate the unit vector between two points
def unit_vector(point1, point2):
    vector = point2 - point1
    norm = np.linalg.norm(vector)
    if norm == 0:
        logging.warning(f"Zero vector detected between points {point1} and {point2}.")
        return np.array([0, 0, 0])
    return vector / norm

# Function to calculate the horizontal perpendicular vector
def horizontal_perpendicular_vector(vector):
    return np.array([-vector[1], vector[0], 0])

# Function to calculate the vertical perpendicular vector
def vertical_perpendicular_vector(_):
    return np.array([0, 0, 1])

# Function to generate tracking points based on the best y points and specified distances
def generate_tracking_points(best_xy, distances):
    TrackingPoints = namedtuple('TrackingPoints', ['horizon_left', 'horizon_right', 'vertical_up', 'vertical_down'])
    tracking_points = TrackingPoints([], [], [], [])
    
    if len(best_xy) == 0:
        logging.warning("Empty best_xy array detected.")
        return tracking_points._asdict()
    
    # Loop through best_xy to generate tracking points
    for i in range(len(best_xy) - 1):
        point1, point2 = best_xy[i], best_xy[i + 1]
        direction_vector = unit_vector(point1[:3], point2[:3])
        horizontal_unit_vector = horizontal_perpendicular_vector(direction_vector)
        vertical_unit_vector = vertical_perpendicular_vector(direction_vector)
        
        tracking_points.horizon_left.append(point1[:3] - distances['horizon_left'] * horizontal_unit_vector)
        tracking_points.horizon_right.append(point1[:3] + distances['horizon_right'] * horizontal_unit_vector)
        tracking_points.vertical_up.append(point1[:3] + distances['vertical_up'] * vertical_unit_vector)
        tracking_points.vertical_down.append(point1[:3] - distances['vertical_down'] * vertical_unit_vector)
    
    # Handle the last point separately to ensure all points are covered
    last_point = best_xy[-1]
    if len(best_xy) > 1:
        previous_point = best_xy[-2]
        direction_vector = unit_vector(last_point[:3], previous_point[:3])
        horizontal_unit_vector = horizontal_perpendicular_vector(direction_vector)
        vertical_unit_vector = vertical_perpendicular_vector(direction_vector)
        
        tracking_points.horizon_left.append(last_point[:3] + distances['horizon_left'] * horizontal_unit_vector)
        tracking_points.horizon_right.append(last_point[:3] - distances['horizon_right'] * horizontal_unit_vector)
        tracking_points.vertical_up.append(last_point[:3] + distances['vertical_up'] * vertical_unit_vector)
        tracking_points.vertical_down.append(last_point[:3] - distances['vertical_down'] * vertical_unit_vector)
    
    return tracking_points._asdict()

# Function to save the generated tracking points to a CSV file
def save_tracking_points(filename, tracking_points):
    df = pd.DataFrame(tracking_points, columns=['x', 'y', 'z'])
    df.to_csv(filename, index=False)
    logging.info(f'Tracking points saved to {filename}.')

# Function to plot the best y points and tracking points in a 3D space
def plot_points(best_xy, tracking_points, filename, tp):
    fig = go.Figure()
    
    # Plot the best y points (critical points)
    fig.add_trace(go.Scatter3d(x=best_xy[:, 0], y=best_xy[:, 1], z=best_xy[:, 2], 
                               mode='markers', marker=dict(size=5, color='black'), name='Critical Points'))
    
    color_dict = {
        'horizon_left': 'green',
        'horizon_right': 'magenta',
        'vertical_up': 'red',
        'vertical_down': 'blue'
    }
    
    # Plot each set of tracking points with the specified color
    for key, points in tracking_points.items():
        if len(points) == 0:
            continue  # Skip if the list is empty
        
        points = np.array(points)
        if points.ndim == 1:  # Skip if the array is 1-dimensional
            continue

        fig.add_trace(go.Scatter3d(x=points[:, 0], y=points[:, 1], z=points[:, 2], 
                                   mode='markers', marker=dict(size=3, color=color_dict[key]), 
                                   name=key.replace('_', ' ').title()))
    
    # Add tp value in the plot title
    fig.update_layout(
        title=f'3D Tracking Points for tp = {tp:.2f}',
        scene=dict(
            xaxis=dict(range=[0, 10000], title='X Axis'),
            yaxis=dict(range=[0, 10000], title='Y Axis'),
            zaxis=dict(range=[-2000, 0], title='Z Axis'),
            aspectmode='manual',
            aspectratio=dict(x=1, y=1, z=0.5)
        ),
        margin=dict(l=0, r=0, b=0, t=0)
    )

    fig.show()
    fig.write_html(str(filename))
    logging.info(f'Plot saved as HTML: {filename} with tp = {tp:.2f}')

# Function to process the CSV file and generate DAT and MCR files
def process_csv_file(file_prefix, color):
    input_path = Path('results') / f'{file_prefix}.csv'
    try:
        df = pd.read_csv(input_path)
    except FileNotFoundError:
        logging.error(f"File not found: {input_path}")
        return
    
    output_path_dat = Path('results') / f'{file_prefix}.dat'
    df[['x', 'y', 'z']].to_csv(output_path_dat, sep='\t', index=False, header=False)
    
    # Add header information to the DAT file
    with open(output_path_dat, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write('VARIABLES = "X", "Y", "Z"\n')
        f.write(f"ZONE T='{file_prefix}'\n")
        f.write(f'I={len(df)}, J=1, K=1, F=POINT\n')
        f.write(content)
    
    # Generate the corresponding macro file for visualization
    generate_macro_file(df, file_prefix, color)
    logging.info(f'Generated files: {output_path_dat}, {file_prefix}.mcr')

# Function to generate a macro file for visualization
def generate_macro_file(df, file_prefix, color):
    macro_file_path = Path('results') / f'{file_prefix}.mcr'
    points = df[['x', 'y', 'z']].values.tolist()
    
    with open(macro_file_path, mode='w') as macrofile:
        macrofile.write('#!MC 1410\n')
        macrofile.write(f'$!StreamAttributes Color = blue\n')
        for point in points:
            macrofile.write('$!Streamtrace Add\n')
            macrofile.write('  StreamType = VolumeLine\n')
            macrofile.write('  StreamDirection = Both\n')
            macrofile.write('  StartPos\n')
            macrofile.write('    {\n')
            macrofile.write(f'    X = {point[0]}\n')
            macrofile.write(f'    Y = {point[1]}\n')
            macrofile.write(f'    Z = {point[2]}\n')
            macrofile.write('    }\n')
        macrofile.write('$!REDRAWALL\n')
    logging.info(f'Macro file saved: {macro_file_path}')

# Main function to process all files and generate outputs
def main():
    result_folder = Path(os.getcwd()) / 'results'
    tracking_distances = {
        'horizon_left': 500,
        'horizon_right': 500,
        'vertical_up': 150,
        'vertical_down': 300
    }

    # Loop through different tp values to process each corresponding file
    for tp in np.arange(0, 1, 0.25):
        file_prefix = f'best_y_tp_{tp:.2f}'
        best_y_data = read_best_y(file_prefix)
        if best_y_data is None:
            continue
        
        tracking_points = generate_tracking_points(best_y_data.to_numpy(), tracking_distances)
        
        # Save tracking points to individual CSV files
        for prefix, points in tracking_points.items():
            save_tracking_points(result_folder / f'best_y_trace_{prefix}_tp_{tp:.2f}.csv', points)
        
        # Print the current tp value for tracking
        print(f'Processing tp = {tp:.2f}')
        
        # Plot the points and save the plot as an HTML file
        #plot_points(best_y_data.to_numpy(), tracking_points, result_folder / f'tracking_points_tp_{tp:.2f}.html', tp)

        # Generate DAT and MCR files for each tracking point category
        for category, color in [('horizon_left', 2), ('horizon_right', 3), ('vertical_up', 4), ('vertical_down', 5)]:
            process_csv_file(f'best_y_trace_{category}_tp_{tp:.2f}', color)

if __name__ == '__main__':
    main()


In [ ]:
# Step 4: Merge Dividing Streamlines and Show Groundwater Flow Systems
import numpy as np
import h5py
import os
import pandas as pd
import plotly.graph_objects as go
from scipy.integrate import solve_ivp
from concurrent.futures import ThreadPoolExecutor
from numba import jit
import plotly.io as pio

# Function to read data from an HDF5 file
def read_data_hdf5(filename):
    with h5py.File(filename, 'r') as f:
        # Read velocity components and other relevant datasets from the file
        vx = f['vx'][:]         # Velocity component in x-direction
        vy = f['vy'][:]         # Velocity component in y-direction
        vz = f['vz'][:]         # Velocity component in z-direction
        UU = f['UU'][:]         # Additional dataset (context-specific)
        x_range = f['x_range'][:]  # Range of x values
        y_range = f['y_range'][:]  # Range of y values
        z_range = f['z_range'][:]  # Range of z values
    return vx, vy, vz, UU, x_range, y_range, z_range

# JIT-compiled function to compute the velocity field at a given position
@jit(nopython=True)
def velocity_field_numba(pos, vx, vy, vz, x_range, y_range, z_range):
    x, y, z = pos
    # Check if the position is outside the defined ranges
    if x < x_range[0] or x > x_range[-1] or y < y_range[0] or y > y_range[-1] or z < z_range[0] or z > z_range[-1]:
        return np.array([0.0, 0.0, 0.0])  # Return zero velocity if out of bounds
    
    # Find the indices of the grid cell containing the position
    xi = np.searchsorted(x_range, x) - 1
    yi = np.searchsorted(y_range, y) - 1
    zi = np.searchsorted(z_range, z) - 1

    # Get the surrounding grid points
    x1, x2 = x_range[xi], x_range[xi+1]
    y1, y2 = y_range[yi], y_range[yi+1]
    z1, z2 = z_range[zi], z_range[zi+1]

    # Compute relative distances within the grid cell
    xd = (x - x1) / (x2 - x1)
    yd = (y - y1) / (y2 - y1)
    zd = (z - z1) / (z2 - z1)

    # Trilinear interpolation for the x-component of velocity
    c00 = vx[xi, yi, zi] * (1 - xd) + vx[xi + 1, yi, zi] * xd
    c01 = vx[xi, yi, zi + 1] * (1 - xd) + vx[xi + 1, yi, zi + 1] * xd
    c10 = vx[xi, yi + 1, zi] * (1 - xd) + vx[xi + 1, yi + 1, zi] * xd
    c11 = vx[xi, yi + 1, zi + 1] * (1 - xd) + vx[xi + 1, yi + 1, zi + 1] * xd

    c0 = c00 * (1 - yd) + c10 * yd
    c1 = c01 * (1 - yd) + c11 * yd

    vx_val = c0 * (1 - zd) + c1 * zd  # Final interpolated x-velocity

    # Trilinear interpolation for the y-component of velocity
    c00 = vy[xi, yi, zi] * (1 - xd) + vy[xi + 1, yi, zi] * xd
    c01 = vy[xi, yi, zi + 1] * (1 - xd) + vy[xi + 1, yi, zi + 1] * xd
    c10 = vy[xi, yi + 1, zi] * (1 - xd) + vy[xi + 1, yi + 1, zi] * xd
    c11 = vy[xi, yi + 1, zi + 1] * (1 - xd) + vy[xi + 1, yi + 1, zi + 1] * xd

    c0 = c00 * (1 - yd) + c10 * yd
    c1 = c01 * (1 - yd) + c11 * yd

    vy_val = c0 * (1 - zd) + c1 * zd  # Final interpolated y-velocity

    # Trilinear interpolation for the z-component of velocity
    c00 = vz[xi, yi, zi] * (1 - xd) + vz[xi + 1, yi, zi] * xd
    c01 = vz[xi, yi, zi + 1] * (1 - xd) + vz[xi + 1, yi, zi + 1] * xd
    c10 = vz[xi, yi + 1, zi] * (1 - xd) + vz[xi + 1, yi + 1, zi] * xd
    c11 = vz[xi, yi + 1, zi + 1] * (1 - xd) + vz[xi + 1, yi + 1, zi + 1] * xd

    c0 = c00 * (1 - yd) + c10 * yd
    c1 = c01 * (1 - yd) + c11 * yd

    vz_val = c0 * (1 - zd) + c1 * zd  # Final interpolated z-velocity

    return np.array([vx_val, vy_val, vz_val])  # Return the interpolated velocity vector

# Function to compute streamlines based on velocity fields and start points
def compute_streamlines(vx, vy, vz, x_range, y_range, z_range, start_points, max_distance=1e7, tol=1e-7):
    # Define the velocity field function for the ODE solver
    def velocity_field(t, pos):
        return velocity_field_numba(pos, vx, vy, vz, x_range, y_range, z_range)

    # Define an event to terminate integration when a boundary condition is met
    def boundary_event(t, pos):
        return pos[2] + 40  # Termination condition, e.g., z = -80 surface

    boundary_event.terminal = True  # Stop integration when event is triggered
    boundary_event.direction = 0     # Event is detected regardless of direction

    # Function to integrate the streamline starting from a given point
    def integrate_streamline(start):
        # Integrate forward in time
        result_forward = solve_ivp(velocity_field, [0, max_distance], start, method='RK45', rtol=tol, atol=tol, events=boundary_event)
        
        # Integrate backward in time
        result_backward = solve_ivp(velocity_field, [0, -max_distance], start, method='RK45', rtol=tol, atol=tol, events=boundary_event)
        
        # Extract the streamline points
        streamline_forward = result_forward.y.T
        streamline_backward = result_backward.y.T
        
        # Reverse the backward streamline and combine with forward streamline
        streamline_backward = streamline_backward[::-1]
        streamline = np.vstack((streamline_backward, streamline_forward))
        
        return streamline

    # Use a thread pool to parallelize streamline computations
    with ThreadPoolExecutor() as executor:
        streamlines = list(executor.map(integrate_streamline, start_points))
    
    return streamlines  # Return the list of computed streamlines

# Function to plot streamlines and their corresponding start points
def plot_streamlines_and_points(fig, streamlines, start_points, color):
    for streamline in streamlines:
        if len(streamline) > 0:
            fig.add_trace(go.Scatter3d(
                x=streamline[:, 0], y=streamline[:, 1], z=streamline[:, 2],
                mode='lines',
                line=dict(color=color, width=4)  # Set streamline color and width
            ))
    
    # Plot the start points of the streamlines
    fig.add_trace(go.Scatter3d(
        x=start_points[:, 0], y=start_points[:, 1], z=start_points[:, 2],
        mode='markers',
        marker=dict(color=color, size=5, symbol='square'),  # Set marker color, size, and shape
        name=f'Start Points {color}'
    ))

# Function to read trace points from a CSV file
def read_trace_points(file_path):
    # Read the first few rows to determine the number of columns
    df = pd.read_csv(file_path, nrows=5)
    
    # Check the number of columns and read the full file accordingly
    if df.shape[1] == 4:
        df = pd.read_csv(file_path, names=['x', 'y', 'z', 'UU'], header=0, skiprows=0)
    elif df.shape[1] == 3:
        df = pd.read_csv(file_path, names=['x', 'y', 'z'], header=0, skiprows=0)
    else:
        raise ValueError("CSV file format is incorrect. It should contain 3 or 4 columns.")
    
    return df[['x', 'y', 'z']].values  # Return only the x, y, z coordinates

# Get the current working directory as the base path for the script
current_working_dir = os.getcwd()

# Define the path to the results folder and create it if it doesn't exist
result_folder = os.path.join(current_working_dir, 'results')
os.makedirs(result_folder, exist_ok=True)

# Define a list of colors to be used for different tracking point categories
colors = [ 'magenta','green', 'red', 'blue']

# Define prefixes for the tracking point files
tracking_point_prefixes = [
    'best_y_trace_horizon_left',
    'best_y_trace_horizon_right',
    'best_y_trace_vertical_up',
    'best_y_trace_vertical_down'
]

# Function to read the "best_y.csv" file containing points of interest
def read_best_y(file_prefix):
    best_y_path = os.path.join(result_folder, f'{file_prefix}.csv')
    df = pd.read_csv(best_y_path, header=0, skiprows=0)
    return df

# Loop through different tp values to process each corresponding file
for tp in np.arange(0, 1, 0.25):  # Iterate over tp from 0 to 0.9 with step 0.1
    # Define the path to the input HDF5 file for the current tp value
    input_path_hdf5 = os.path.join(result_folder, f'head_and_velocity_tp_{tp:.2f}.h5')
    
    # Read data from the HDF5 file
    vx, vy, vz, UU, x_range, y_range, z_range = read_data_hdf5(input_path_hdf5)

    # Create a new Plotly figure for visualization
    fig = go.Figure()

    # Loop through each tracking point category and its corresponding color
    for prefix, color in zip(tracking_point_prefixes, colors):
        # Define the path to the tracking point CSV file
        file_path = os.path.join(result_folder, f'{prefix}_tp_{tp:.2f}.csv')
        # Read the start points for streamlines
        start_points = read_trace_points(file_path)
        
        # Compute streamlines based on the velocity field and start points
        streamlines = compute_streamlines(vx, vy, vz, x_range, y_range, z_range, start_points)
        
        # Plot the computed streamlines and their start points on the figure
        plot_streamlines_and_points(fig, streamlines, start_points, color)

    # Define the path to the "best_y" points CSV file for the current tp value
    best_y_path = os.path.join(result_folder, f'best_y_tp_{tp:.2f}.csv')
    # Read the best y points
    best_y_points = read_trace_points(best_y_path)

    # Add the best y points to the plot as black hollow circles
    fig.add_trace(go.Scatter3d(
        x=best_y_points[:, 0], y=best_y_points[:, 1], z=best_y_points[:, 2],
        mode='markers',
        marker=dict(color='black', size=6, symbol='circle'),  # Black hollow circles
        name='Best Y Points'
    ))

    # Update the layout of the plot to set axis ranges and aspect ratios
    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[0, 10000], title='X Axis'),  # Set X-axis range and title
            yaxis=dict(range=[0, 10000], title='Y Axis'),  # Set Y-axis range and title
            zaxis=dict(range=[-2000, 0], title='Z Axis'),  # Set Z-axis range and title
            aspectmode='manual',  # Enable manual aspect ratio settings
            aspectratio=dict(x=1, y=1, z=0.5)  # Define custom aspect ratios
        ),
        margin=dict(l=0, r=0, b=0, t=0)  # Remove margins around the plot
    )
    
    # Print the current tp value for tracking
    print(f'Processing tp = {tp:.2f}')
    
    # Define the path to save the output HTML file for the current tp value
    output_html_file = os.path.join(result_folder, f'3D_GFSs_Case2T_taud_1_tp_{tp:.2f}.html')
    # Save the figure as an HTML file without automatically opening it
    pio.write_html(fig, file=output_html_file, auto_open=False)

    # Display the figure
    fig.show()
